In [9]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

seat_data = pd.read_csv('SeatPlan.csv') # Everyones seat preferences 

seat_data['Extra1'] = seat_data['Extra1'].fillna('None')
seat_data['Extra2'] = seat_data['Extra2'].fillna('None')
seat_data['Extra3'] = seat_data['Extra3'].fillna('None')
seat_data['Extra4'] = seat_data['Extra4'].fillna('None')
seat_data['Extra5'] = seat_data['Extra5'].fillna('None')
seat_data['Extra6'] = seat_data['Extra6'].fillna('None')

def shuffle_lonely_people(group):
    if group['Extras'].iloc[0] == 'None':
        group['Extras'] = np.random.permutation(group['Extras'].values)
    return group

seat_data = seat_data.groupby(['Plus 1s']).apply(shuffle_lonely_people)

smalltablecapacity = 8
bigtablecapacity = 10

table_assignments = []

# Iterate through attendees and assign them to tables
for index, attendee in seat_data.iterrows():
    assigned = False
    for table, guests in table_assignments.items():
        # Check if there is room for the attendee at the table
        if len(guests) + 1 <= bigtablecapacity:
            # Check if the attendee has specified friends
            if attendee['Extras'] in guests or attendee['Extras'] == 'None':
                table_assignments[table].append(attendee['First Name'] + ' ' + attendee['Last Name'])
                assigned = True
                break
    # If no suitable table found, create a new table
    if not assigned:
        table_assignments[len(table_assignments) + 1] = [attendee['First Name'] + ' ' + attendee['Last Name']]

# Print table assignments
for table, guests in table_assignments.items():
    print("Table", table, ":", guests)



KeyError: 'Extras'

In [ ]:

X = seat_data.drop(columns=['First Name', 'Last Name', 'Plus 1s'], axis=1)
y = seat_data['Plus 1s']

clf = RandomForestClassifier(n_estimators=100, random_state=42)

clf.fit(X,y)

# Define hyperparameters grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1)

# Perform Grid Search to find the best parameters
grid_search.fit(X, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.2)

# model = DecisionTreeClassifier() 
# model.fit(X, y)

# tree.export_graphviz(clf, out_file='seat-plan.dot',
#                      feature_names=['age', 'gender'],
#                      class_names=sorted(y.unique()),
#                      label='all',
#                      rounded=True,
#                      filled=True)

# joblib.load('music-recommender.joblib')
# predictions = model.predict([[21,1]])

# score = accuracy_score(y_test, predictions)
# score